In [1]:
%reset -f
%run helper.ipynb
%matplotlib widget
%jsroot on

a = Ana("resoCorrFront", "run530164", 1)
p = Par(a.runName)
c = Canva()

Welcome to JupyROOT 6.26/06

 Beginning new ROOT session with private TStyle 


In [2]:
c.new()
hq1 = a.geth(F"{p.q1}", "250,0,500", p.fidLarge, "")
hq0 = a.geth(F"{p.q0}", "250,0,500", p.fidLarge, "same")
hq1.SetLineColor(kRed)
cCutSipm = "charge[0] > 50 && charge[1] > 50"

In [3]:
c.new()
htmq = a.geth(F"{p.dtSipm}:{p.mcSipm}", "1000,0,500,1000,-1,1", "", "zcol")
FormatH(htmq, a.runName, "mean charge [pC]", "raw T_{diff} [ns]", 0)
c.savelast(a.saveas("before"))

singlePop = 0
popCut = "1"
popTag = ""

def cutdt(dt):
    return 1

if (singlePop != 0):
    popTag = '_up' if (singlePop == 1) else ('_down' if (singlePop == 2) else ' ')
    popCut = F"{p.dtSipm} {'> 0.2' if (singlePop==1) else '< -0.2'}"
    c.new()
    htmqSing = a.geth(F"{p.dtSipm}:{p.mcSipm}", "1000,0,500,1000,-1,1", popCut, "zcol")

    def cutdt(dt):
        return ((dt>0.2)&(singlePop==1) or (dt<-0.2)&(singlePop==2) )


Info in <TCanvas::Print>: pdf file out/run530164/resoCorrFront/before.pdf has been created


In [4]:
c.new()
hcorr = a.geth(F"{p.dtSipm}:{p.asymSipm}", "1000,-0.2,0.2,1000,-1,1", p.fidLarge +p.cutAnd+ cCutSipm +p.cutAnd+ popCut, "zcol")
FormatH(hcorr, a.runName, "(Q1-Q0)/(Q1+Q0)", "T1 - T0 [ns]", 0)
if (singlePop == 0): c.savelast(a.saveas("hcorr"))

profcorr = hcorr.ProfileX()
FormatH(profcorr, a.runName, "(Q1-Q0)/(Q1+Q0)", "raw T_{diff} profile [ns]", 0)
profcorr.Rebin(4)
profcorr.GetXaxis().SetRangeUser(-0.15, 0.1)
profcorr.SetMarkerSize(.2)
profcorr.SetMarkerStyle(20)

# c.new()
# profcorr.Draw()
# if (singlePop == 0): c.savelast(a.saveas("corrProf"))

asymRng = [-0.1, 0.04]
asymRng = [-0.1, -0.04] if (singlePop == 1) else asymRng
asymRng = [-0.02, 0.04] if (singlePop == 2) else asymRng
asymCut = F"{p.asymSipm} > {asymRng[0]} && {p.asymSipm} < {asymRng[1]}"

splincorr = TSpline3(profcorr)

c.new()
profcorr.GetXaxis().SetRangeUser(asymRng[0], asymRng[1])
splincorr.SetLineColor(kRed)
profcorr.Draw("")
splincorr.Draw("L same")
if (singlePop == 0): c.savelast(a.saveas("corrSpline"))

Info in <TCanvas::Print>: pdf file out/run530164/resoCorrFront/hcorr.pdf has been created
Info in <TCanvas::Print>: pdf file out/run530164/resoCorrFront/corrSpline.pdf has been created


In [5]:
histoS = TH2D("histo", "histo", 1000, 0, 500, 1000, -1, 1)
prova = TH2D("prova", "prova", 200, p.fidlarge[2], p.fidlarge[3], 100, -1, 1)
prova2 = TH2D("prova2", "prova2", 200, p.fidlarge[0], p.fidlarge[1], 200, -1, 1)

FormatH(histoS, "corrected deltaT/2 [ns]", "mean charge [pC]", "deltaT/2 [ns]", 0)
FormatH(prova, "", "track Y [cm]", "t1 - t0 [ns]", 0)
FormatH(prova2, "", "track X [cm]", "t1 - t0 [ns]", 0)

for i in tqdm(range(0, a.tree.GetEntries())):
    a.tree.GetEntry(i)
    dt = a.tree.tim[1] - a.tree.tim[0]
    if (cutdt(dt)==False): continue
    asy = (a.tree.charge[1]-a.tree.charge[0])/(a.tree.charge[1]+a.tree.charge[0])
    val = dt - splincorr.Eval(asy)
    val *= 0.5
    mq = 0.5* (a.tree.charge[1]+a.tree.charge[0])
    histoS.Fill(mq, val)
    prova.Fill(a.tree.cryPos[1], val)
    prova2.Fill(a.tree.cryPos[0], val)


100%|██████████| 99039/99039 [00:56<00:00, 1743.58it/s]


In [6]:
c.new(500,300)
prova.Draw("zcol")
c.new(500,300)
prova2.Draw("zcol")

In [7]:
c.new()
histoS.Draw("zcol")
c.savelast(a.saveas("after"+popTag))

Info in <TCanvas::Print>: pdf file out/run530164/resoCorrFront/after.pdf has been created


In [10]:
histo = histoS.Clone()

nfitpar = 2

eMin = 50
eSteps = 6
eMax = 300
steps = np.linspace(eMin, eMax, eSteps+1)
stepsNo = steps.shape[0]

fout = TFile(a.outPath + F"resoCorr{popTag}.root", "recreate")

gra = TGraphErrors("gra")
FormatH(gra, a.runName, "mean SiPM charge [pC]", "mean time cell resolution [ns]", 0)

for i, thisEne in enumerate(steps):
    if(i+1 == stepsNo): break
    sliceName = F"slice_{i}"
    histo.GetXaxis().SetRangeUser(steps[i], steps[i+1])
    slice = histo.ProjectionY()
    fitf = TF1("fitf", "gaus", -1, 1)
    fitf.SetParameter(1, slice.GetBinCenter(slice.GetMaximumBin()))
    fitf.SetParameter(2, 0.2*slice.GetRMS())
    slice.Fit(fitf, "REMQ")
    charg = histo.ProjectionX()
    FillGraph(gra, charg.GetMean(), fitf.GetParameter(2), charg.GetRMS()/math.sqrt(charg.Integral()), fitf.GetParError(2))
    fout.cd()
    slice.Write()
    charg.Write()
    slice.Delete()

gra.Write()
gra.SetMarkerStyle(20)
gra.SetMarkerSize(.5)

c.new()
gra.SetStats(1)
gStyle.SetOptStat(1)
gStyle.SetOptFit(11)
gStyle.SetFitFormat("g")

fitf=TF1()
if (nfitpar==3):
    fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1]/x + [2]*[2])")
    fitf.SetParNames("a [ns#bulletpC]", "b [ns#bulletpC^{0.5}]", "c [ns]")
else:
    fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1])")
    fitf.SetParNames("a [ns#bulletpC]", "b [ns]")

r = gra.Fit(fitf)

qFrom, qTo = 50, 300
scalenpe = 2.48
npeHeigth = 7e-3

gra.SetMinimum(npeHeigth)
gra.GetXaxis().SetRangeUser(qFrom, qTo)
fnpe = TF1("fnpe", F"x*{scalenpe}")
fnpe.SetRange(fnpe.Eval(qFrom+10), fnpe.Eval(qTo-10))
anpe = TGaxis(qFrom+10, 2*npeHeigth, qTo-10, 2*npeHeigth, "fnpe", 510, "-")
anpe.SetTitle("N_{pe}")
anpe.SetTitleOffset(-1.1)
anpe.SetLabelSize(0.03)
color = kBlue
anpe.SetLineColor(color)
anpe.SetLabelColor(color)
anpe.SetTextColor(color)
#anpeSetTitleSize(0.06)

gra.Draw("AP")
anpe.Draw("same")

savename = F"resocorr_{nfitpar}par{popTag}"
c.savelast(a.saveas(savename))
c.savelast(a.saveas(savename, ".C"))

fout.Close()

 FCN=25.2504 FROM MIGRAD    STATUS=CONVERGED      83 CALLS          84 TOTAL
                     EDM=6.22549e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  a [ns#bulletpC]   3.03649e+00   2.55963e-02   4.16719e-05   2.46391e-03
   2  b [ns]       2.23033e-02   1.61883e-04   2.63553e-07  -3.45168e-01


Error in <TGraphErrors::TGraphErrors>: Cannot open file: gra, TGraphErrors is Zombie
Info in <TCanvas::Print>: pdf file out/run530164/resoCorrFront/resocorr_2par.pdf has been created
Info in <TCanvas::SaveSource>: C++ Macro file: out/run530164/resoCorrFront/resocorr_2par.C has been generated
